# Импортирање на потребните библиотеки

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.8 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 40.5 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
     |████████████████████████████████| 636 kB 31.3 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 5.4 MB/s 
     |████████████████████████████████| 243 kB 39.0 MB/s 
     |████████████████████████████████| 119 kB 48.2 MB/s 


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from datasets import load_dataset
import pandas as pd
import numpy as np

# Вчитување податоци

In [ ]:
# го вчитуваме податочното множество IMDB кое е составено од reviews
# на филмови означени како позитивни или негативни
data = load_dataset('imdb')

# за полесно користење на податоците, ги трансформираме
# во pandas DataFrame објекти
train = pd.DataFrame().from_dict(data['train'])
test = pd.DataFrame().from_dict(data['test'])

# за побрзо извршување на показните примери, користиме само дел
# од податочното множество
train = train[:2000]
test = test[:1000]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


In [ ]:
# приказ на дел од записите во податочното множество
train.head()

,text,label
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


# BERT

## Претпроцесирање на податоци

In [ ]:
# за трансформација на текстуалните податоци во формат соодветен за моделот
# BERT ќе користиме BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# дефинираме празни листи за идентификатори на зборовите во секоја реченица,
# маските на внимание и класите
train_input_ids, train_attention_masks, train_outputs = [], [], []
for sentence, label in zip(train['text'].values, train['label'].values):
    # секоја реченица се трансформира во потребниот формат
    # со аргументот max_length се дефинира максималната должина на реченицата
    # со аргументот pad_to_max_length се означува дека ако реченицата е 
    # пократка од максималната должина, тогаш таа се дополнува со токени до
    # посакуваната должина
    # со аргументот truncation се означува дека ако реченицата е подолга од
    # максималната должина, тогаш се отстрануваат токени се додека реченицата
    # не ја добие посакуваната должина
    sentence_tokens = bert_tokenizer.encode_plus(sentence, max_length=30, pad_to_max_length=True, truncation=True)
    train_input_ids.append(sentence_tokens['input_ids'])
    train_attention_masks.append(sentence_tokens['attention_mask'])

    # ги трансформираме класите во one-hot вектори
    new_label = [0, 1] if label == 0 else [1, 0]
    train_outputs.append(new_label)

In [ ]:
# дефинираме празни листи за идентификатори на зборовите во секоја реченица,
# маските на внимание и класите
test_input_ids, test_attention_masks, test_outputs = [], [], []
for sentence, label in zip(test['text'].values, test['label'].values):
    # секоја реченица се трансформира во потребниот формат
    # со аргументот max_length се дефинира максималната должина на реченицата
    # со аргументот pad_to_max_length се означува дека ако реченицата е 
    # пократка од максималната должина, тогаш таа се дополнува со токени до
    # посакуваната должина
    # со аргументот truncation се означува дека ако реченицата е подолга од
    # максималната должина, тогаш се отстрануваат токени се додека реченицата
    # не ја добие посакуваната должина
    sentence_tokens = bert_tokenizer.encode_plus(sentence, max_length=30, pad_to_max_length=True, truncation=True)
    test_input_ids.append(sentence_tokens['input_ids'])
    test_attention_masks.append(sentence_tokens['attention_mask'])

    # ги трансформираме класите во one-hot вектори
    new_label = [0, 1] if label == 0 else [1, 0]
    test_outputs.append(new_label)

## Креирање модели

In [ ]:
# за проблем на класификација со моделот BERT може да се користи
# TFBertForSequenceClassification
# при инстанцирање на моделот се дефинира која верзија на моделот BERT ќе се
# користи (bert-base-uncased - верзија на BERT моделот со 12 слоеви тренирана 
# без правење разлика помеѓу мали и големи букви) и бројот на класи (2 за
# класификација на позитивен и негативен сентимент)
bert_classification_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# пред да се тренира, потребно е моделот да се компајлира
# со поставување на аргументот loss дефинираме категориска крос-ентропија
# како функција на загуба
# со поставување на аргументот optimizer дефинираме Adam оптимизатор со рата
# на учење еднаква на 0.01
# со поставување на аргументот metrics дефинираме точност како метрика за 
# следење на перформансите на моделот при тренирање
bert_classification_model.compile(optimizer=Adam(learning_rate=0.01), loss=categorical_crossentropy, metrics=['accuracy'])

## Тренирање и евалуација

In [ ]:
# при тренирање на моделот покрај влезните и излезните податоци, потребно е
# да се постави вредност за аргументот epochs што претставува број на епохи
bert_classification_model.fit([np.array(train_input_ids), np.array(train_attention_masks)],
                              np.array(train_outputs), 
                              epochs=3)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [ ]:
# евалуација на моделот со што се добиваат вредности за
# функцијата на загуба и точноста
bert_classification_model.evaluate([np.array(test_input_ids), np.array(test_attention_masks)],
                                   np.array(test_outputs))

32/32 [==============================] - 8s 159ms/step - loss: 1.1921e-07 - accuracy: 1.0000


[1.1920930376163597e-07, 1.0]